# Answer Coalescence

Strider finds and ranks all paths that fulfill the query.  Some of these answers can be very similar.  For instance, two answers may differ only by a gene node, but perhaps those genes are from the same family.  This permits a higher-level, more parsimonious description of an answer set.

In [105]:
import requests
import json

class Strider:
    def __init__(self):
        self.url='http://robokop.renci.org:5781'
    def call(self,question):
        message = {'message': {'query_graph': question}}
        return self.send_message(message)
    def send_message(self,message):
        response = requests.post(f'{self.url}/query',json=message)
        if response.status_code == 200:
            pid = response.json()
            return pid
        else:
            print(response.status_code)
            return None
    def query_result(self,pid):
        r = requests.get(f'{self.url}/results',params={'query_id':pid})
        return r.json()
    
strider = Strider()

In [106]:
disease = 'MONDO:0005180'  #Parkinson
#disease = 'MONDO:0004979'  #Asthma
#disease = "MONDO:0005148"   #T2D
edgetype="contributes_to"
question = { "nodes": [{"id":"n0", "type":"disease", "curie":disease},
                      {"id": "n1", "type":"chemical_substance"}],
             "edges": [ {"id": "e0", "source_id": "n1", "target_id": "n0", "type": edgetype}]}
message = {"message": {"query_graph": question}}
print(message)

{'message': {'query_graph': {'nodes': [{'id': 'n0', 'type': 'disease', 'curie': 'MONDO:0005180'}, {'id': 'n1', 'type': 'chemical_substance'}], 'edges': [{'id': 'e0', 'source_id': 'n1', 'target_id': 'n0', 'type': 'contributes_to'}]}}}


In [107]:
p = strider.send_message(message)
print(p)

713c9b65-ea5e-48a4-a219-d9dc35eb9079


In [111]:
result=strider.query_result(p)
print(len(result['results']))

0


In [53]:
for n in result['knowledge_graph']['nodes']:
    print(n['id'], n['name'])

MONDO:0005180 Parkinson disease
MESH:C084924 1,2,3,4-tetrahydro-1-(phenylmethyl)isoquinoline
MESH:D000661 Amphetamine
MESH:D052638 Particulate Matter
MESH:D008062 Lipofuscin
MESH:D002791 Cholesterol, Dietary
MESH:D063086 Organothiophosphates
MESH:D026023 Permethrin
CHEBI:39300 propargite
MESH:D008054 Lipid Peroxides
MESH:D009569 Nitric Oxide
MESH:D000535 Aluminum
MESH:D005659 Fungicides, Industrial
MESH:C014121 neuromelanin


## Coalesce Answers by property

In [104]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'property'}
message = {'message': res}
print (res.keys())
r = requests.post(url,json=res,params=params)
print(r.status_code)
#print(json.dumps(r.json(),indent=4))

dict_keys(['query_graph', 'results', 'knowledge_graph'])
500


In [8]:
print(json.dumps(r.json(),indent=4))

{
    "query_graph": {
        "nodes": [
            {
                "id": "n0",
                "curie": "MONDO:0005180",
                "type": "disease"
            },
            {
                "id": "n1",
                "curie": null,
                "type": "chemical_substance"
            }
        ],
        "edges": [
            {
                "id": "e0",
                "type": "contributes_to",
                "source_id": "n1",
                "target_id": "n0"
            }
        ]
    },
    "knowledge_graph": {
        "nodes": [
            {
                "id": "MONDO:0005180",
                "name": "Parkinson disease",
                "type": [
                    "named_thing",
                    "disease",
                    "disease_or_phenotypic_feature"
                ],
                "equivalent_identifiers": [
                    "UMLS:C0030567",
                    "UMLS:C0270729",
                    "UMLS:C0542165",
                   

In [8]:
question = { 'nodes': [{'id':'n0', 'type':'disease'},
                      {'id': 'n1', 'type':'chemical_substance', 'curie': 'CHEBI:6801'}],
             'edges': [ {'id': 'e0', 'source_id': 'n1', 'target_id': 'n0', 'type': 'treats'}]}
message = {'message': {'query_graph': question}}

In [9]:
p = strider.send_message(message)

In [11]:
strider.query_result(p)

{
  "query_graph": {
    "nodes": [
      {
        "id": "n1",
        "curie": "CHEBI:6801",
        "type": "chemical_substance"
      },
      {
        "id": "n0",
        "curie": null,
        "type": "disease"
      }
    ],
    "edges": [
      {
        "id": "e0",
        "type": "treats",
        "source_id": "n1",
        "target_id": "n0"
      }
    ]
  },
  "knowledge_graph": null,
  "results": []
}
